<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
n_documents = corpus.shape[0]
d = {}
for i in range(n_documents):
    d[f"document{i}"] = corpus[i].split()

In [5]:
list(d.values())

[['que', 'dia', 'es', 'hoy'],
 ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'],
 ['martes', 'muchas', 'gracias']]

In [6]:
bag_of_words = []
for i in d.keys():
  bag_of_words += d[i]

In [7]:
set(bag_of_words)

{'de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'}

In [8]:
np.unique(bag_of_words)

array(['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas',
       'que'], dtype='<U7')

In [9]:
# Opción vista en clase 
corpus_terminos = []
for doc in corpus:
  terminos = doc.split(" ")
  corpus_terminos.append(terminos)

vocab_completo = np.sum(corpus_terminos)
vocab = np.unique(vocab_completo)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [10]:
vocab

array(['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas',
       'que'], dtype='<U7')

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [11]:
# Opción con for
def OHE(x,vocab):
  vector_ohe = np.zeros(len(vocab))
  for i in range(len(vocab)):
    if vocab[i] in x:
      vector_ohe[i] = 1
  return vector_ohe

# Opción con list comprehension
def OHE_lc(x,vocab):
  vector_ohe = [1 if i in x else 0 for i in vocab ]
  vector_ohe = np.array(vector_ohe)
  return vector_ohe

In [12]:
x=np.array(['juan', 'juan', 'dia', 'dia', 'es', 'hoy'])
print(OHE(x,vocab))
print(OHE(x,vocab).shape)

[0. 1. 0. 1. 0. 1. 0. 0. 0.]
(9,)


In [13]:
print(OHE_lc(x,vocab))
print(OHE_lc(x,vocab).shape)

[0 1 0 1 0 1 0 0 0]
(9,)


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [14]:
from collections import Counter
Counter(x)

Counter({'dia': 2, 'es': 1, 'hoy': 1, 'juan': 2})

In [15]:
def FreqVector(x,vocab):
  c = Counter(x)
  vector_frec = [c[i] for i in vocab ]
  vector_frec = np.array(vector_frec)
  return vector_frec

In [16]:
print(FreqVector(x,vocab))
print(FreqVector(x,vocab).shape)

[0 2 0 1 0 1 0 0 0]
(9,)


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [17]:
N = corpus.shape[0]
ohe = np.zeros((corpus.shape[0],len(vocab)))
TF = np.zeros((corpus.shape[0],len(vocab)))
i = 0
for doc in corpus_terminos:
  ohe[i] = OHE_lc(doc,vocab)
  TF[i] = FreqVector(doc,vocab)
  i += 1

DF = np.sum(ohe,axis=0)
IDF = np.log10(N/DF)
TF_IDF = TF*IDF
print(IDF)
print(TF)
print(TF_IDF)

[0.47712125 0.17609126 0.47712125 0.17609126 0.47712125 0.17609126
 0.17609126 0.47712125 0.47712125]
[[0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 2. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0.]]
[[0.         0.17609126 0.         0.17609126 0.         0.17609126
  0.         0.         0.47712125]
 [0.47712125 0.17609126 0.47712125 0.17609126 0.         0.17609126
  0.35218252 0.         0.        ]
 [0.         0.         0.         0.         0.47712125 0.
  0.17609126 0.47712125 0.        ]]


In [20]:
TF_IDF.shape[0]

3

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [61]:
def compare_docs(corpus, indice):
  corpus_terminos = []
  for doc in corpus:
    terminos = doc.split(" ")
    corpus_terminos.append(terminos)

  vocab_completo = np.sum(corpus_terminos)
  vocab = np.unique(vocab_completo)

  ohe = np.zeros((corpus.shape[0],len(vocab)))
  TF = np.zeros((corpus.shape[0],len(vocab)))
  i = 0
  for doc in corpus_terminos:
    ohe[i] = OHE_lc(doc,vocab)
    TF[i] = FreqVector(doc,vocab)
    i += 1

  DF = np.sum(ohe,axis=0)
  IDF = np.log10(N/DF)
  TF_IDF = TF*IDF

  cos_sim = np.zeros(shape=(TF_IDF.shape[0]-1,TF_IDF.shape[1]))
  i=0
  for j in range(TF_IDF.shape[0]):
    if j != indice:
      cos_sim[i] = cosine_similarity(TF_IDF[indice], TF_IDF[j])
      i += 1
  cos_sim_ordered = np.sort(cos_sim, axis=0)[::-1]
  
  return cos_sim_ordered

In [62]:
compare_docs(corpus,1)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


array([[0.2003419 , 0.2003419 , 0.2003419 , 0.2003419 , 0.2003419 ,
        0.2003419 , 0.2003419 , 0.2003419 , 0.2003419 ],
       [0.10845712, 0.10845712, 0.10845712, 0.10845712, 0.10845712,
        0.10845712, 0.10845712, 0.10845712, 0.10845712]])